In [1]:
from vtk import *
import itk

In [2]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

base_image = itk.ImageFileReader(FileName="./data/BRATS_HG0015_T1C.mha")

rescaled = itk.RescaleIntensityImageFilter.New(
    Input=base_image,
    OutputMinimum=0,
    OutputMaximum=255
)

binary_im = itk.ThresholdImageFilter.New(
    Input=rescaled,
    Lower=102,
)

connected_components = itk.ConnectedComponentImageFilter.New(
    Input=binary_im,
)

filtered_cc = itk.LabelShapeKeepNObjectsImageFilter.New(
    Input=connected_components,
    BackgroundValue=0,
    NumberOfObjects=1,
    Attribute="NumberOfPixels"
)

mask = itk.NotImageFilter(Input=filtered_cc)
mask = itk.NotImageFilter(Input=mask)

converted = itk.CastImageFilter[itk.Image[itk.SS,3], itk.Image[itk.UC,3]].New(Input=mask)

result_im = itk.RescaleIntensityImageFilter.New(
    Input=converted,
    OutputMinimum=0,
    OutputMaximum=1,
)

writer = itk.ImageFileWriter.New(Input=result_im, FileName="./data/mask.mha")
writer.Update()

array = itk.GetArrayFromImage(result_im)

In [3]:
np.unique(array, return_counts=True)

(array([0, 1], dtype=uint8), array([6036216,   46344]))

In [78]:
from vtk import *
import itk

def _check_valid_arg(val, name, available):
    if val not in available:
        raise f"{name}='{val}' is not a valid arg. Valid values are: {available}"
        
def load_volume(reader, color=(1.,1.,1.), render_with="gl", interpolation="linear"):
    _check_valid_arg(render_with, "render_with", {'gl', 'gpu', 'cpu'})
    _check_valid_arg(interpolation, "interpolation", {'linear', 'nearest'})
    
    reader.Update()

    if render_with == "gl":
        mapper = vtkOpenGLGPUVolumeRayCastMapper()
    elif render_with == 'gpu':
        mapper = vtkGPUVolumeRayCastMapper() 
    elif render_with == 'cpu':
        mapper = vtkFixedPointVolumeRayCastMapper()
    else:
        raise "unexpected"
        
    mapper.SetInputConnection(reader.GetOutputPort())
    mapper.SetAutoAdjustSampleDistances(0)
    mapper.SetSampleDistance(0.5)
    mapper.SetMaskTypeToLabelMap()
    mapper.SetMaskBlendFactor(0.7)
    mapper.SetBlendModeToComposite()
    
    props = vtkVolumeProperty()
    props.SetIndependentComponents(True) 
    props.ShadeOff()

    if interpolation == "linear":
        props.SetInterpolationTypeToLinear()
    elif interpolation == 'nearest':
        props.SetInterpolationTypeToNearest()
    else:
        raise "unexpected"

    volume = vtkVolume()
    volume.SetMapper(mapper)
    volume.SetProperty(props)
    
    return volume

In [79]:
reader_brain = vtkMetaImageReader()
reader_brain.SetFileName("./data/BRATS_HG0015_T1C.mha")
reader_mask = vtkMetaImageReader()
reader_mask.SetFileName("./data/mask.mha")
reader_mask.Update()

In [80]:
volume = load_volume(reader_brain)
volume_property = volume.GetProperty()
volume_mapper = volume.GetMapper()

In [81]:
data_min, data_max = reader_brain.GetOutput().GetScalarRange()

seg_min, seg_max = 0, 0.6 * data_max
fct_color_default = vtkColorTransferFunction()
fct_color_default.AddRGBSegment(seg_min, *(0,0,0),
                                seg_max, *(1,1,1))

fct_color_mask = vtkColorTransferFunction()
fct_color_mask.AddRGBSegment(seg_min, *(0,0,0),
                             seg_max, *(1,0,0))   

fct_opacity_default = vtkPiecewiseFunction()
fct_opacity_default.AddSegment(seg_min, 0.,
                               seg_max, 0.1)

fct_opacity_mask = vtkPiecewiseFunction()
fct_opacity_mask.AddSegment(seg_min, 0.,
                            seg_max, 1.)

volume_property.SetColor(fct_color_default)
volume_property.SetScalarOpacity(fct_opacity_default)
volume_property.SetLabelColor(1, fct_color_mask)
volume_property.SetLabelScalarOpacity(1, fct_opacity_mask)

In [82]:
volume_mapper.SetMaskInput(reader_mask.GetOutput())

In [83]:
def AddSlider(interactor, value_range, x, y, length=0.25, title="", default_value=None, callback=lambda x: _):
    assert 0 <= x <= 1 and 0 <= y <= 1
    def _cb(s, *args):
        x = s.GetSliderRepresentation().GetValue()
        callback(x)
        
    slider = vtkSliderRepresentation2D()
    slider.SetMinimumValue(value_range[0])
    slider.SetMaximumValue(value_range[-1])
    slider.SetValue(value_range[0] if default_value is None else default_value)
    slider.SetTitleText(title)
    slider.ShowSliderLabelOn()
    
    slider.GetPoint1Coordinate().SetCoordinateSystemToNormalizedDisplay();
    slider.GetPoint1Coordinate().SetValue(x, y);
    slider.GetPoint2Coordinate().SetCoordinateSystemToNormalizedDisplay();
    slider.GetPoint2Coordinate().SetValue(x + length, y);

    sliderWidget = vtkSliderWidget()
    sliderWidget.SetInteractor(interactor);
    sliderWidget.SetRepresentation(slider);
    sliderWidget.EnabledOn();
    
    sliderWidget.AddObserver("InteractionEvent", _cb)
    
    return sliderWidget
    
def OnClose(interactor, event):
    interactor.GetRenderWindow().Finalize()
    interactor.TerminateApp()

In [85]:
def cb_opacity_brain(x):
    fct_opacity_default.AddSegment(seg_min, 0., seg_max, x)
    
def cb_opacity_mask(x):
    volume_mapper.SetMaskBlendFactor(x)
    
ren = vtkRenderer()
ren.AddVolume(volume)

renWin = vtkRenderWindow()
renWin.AddRenderer(ren)

iren = vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

iren.AddObserver('ExitEvent', OnClose)

sl_0 = AddSlider(interactor=iren, value_range=(0, 1), x=0.7, y=0.35, title="Tumor Highlight", default_value=0.7, callback=cb_opacity_mask)
sl_1 = AddSlider(interactor=iren, value_range=(0, 1), x=0.7, y=0.15, title="Brain Opacity", default_value=0.1, callback=cb_opacity_brain)

iren.Initialize()
renWin.Render()
iren.Start()